In [70]:
import warnings
warnings.filterwarnings('ignore')

# 회원 탈퇴 예측

### 1 : 데이터 읽고 이용데이터 수정

In [71]:
# use_log_months.csv & customer_join.csv
import pandas as pd

customer = pd.read_csv('./data/customer_join.csv')
uselog_months = pd.read_csv('./data/use_log_months.csv')

In [72]:
customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaN,CA1,0,종일,10500,일반,8.000000,8.0,8,8,0,2019-04-30,1
4188,HD246549,XXXXX,C01,F,2019-03-14,NaN,CA1,0,종일,10500,일반,10.000000,10.0,10,10,0,2019-04-30,1
4189,GD037007,XXXXX,C03,M,2019-03-14,NaN,CA1,0,야간,6000,일반,8.000000,8.0,8,8,0,2019-04-30,1
4190,OA953150,XXXXX,C01,M,2019-03-14,NaN,CA1,0,종일,10500,일반,11.000000,11.0,11,11,0,2019-04-30,1


In [73]:
uselog_months

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [74]:
uselog_months

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [75]:
# 이번달과 1개월 전의 이용 횟수를 집계
year_months = list(uselog_months["연월"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    # print(year_months[i], year_months[i-1])
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-1]]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    del tmp_before["연월"]
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
    
uselog.head()   

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### 2 : 탈퇴전월의 탈퇴고객데이터 작성

In [76]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
exit_customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,종일,10500,일반,3.0,3.0,3,3,0,2018-04-30,23,None
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,주간,7500,일반,3.0,3.0,3,3,0,2018-04-30,23,None
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,종일,10500,일반,1.0,1.0,1,1,0,2018-04-30,23,None
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,야간,6000,일반,3.5,3.5,4,3,1,2018-05-31,23,None
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,종일,10500,일반,3.0,3.0,3,3,0,2018-05-31,23,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,OA143589,XXXXX,C03,F,2019-01-15,2019-02-28,CA1,1,야간,6000,일반,4.5,4.5,6,3,0,2019-02-28,1,None
4050,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,종일,10500,일반,7.0,6.0,12,3,0,2019-03-31,2,None
4068,OA034442,XXXXX,C02,M,2019-02-05,2019-03-31,CA1,1,주간,7500,일반,7.0,7.0,11,3,0,2019-03-31,1,None
4086,PL508278,XXXXX,C03,M,2019-02-09,2019-03-31,CA1,1,야간,6000,일반,7.0,7.0,10,4,0,2019-03-31,1,None


In [77]:
exit_customer["end_date"].iloc[0]

Timestamp('2018-04-30 00:00:00')

In [78]:
uselog

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


In [79]:
from dateutil.relativedelta import relativedelta

# 탈퇴한 고객
# exit_customer = # customer 의 is_deleted
# exit_date 컬럼을 만들어서 

# end_date의 1개월 전을 계산
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
    
# exit_date 년월로 변경
exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)                                                                             
# uselog와 exit_customer를 merge : 기준이 customer_id, 연월
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")

print(len(uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### 3 : 지속회원의 데이터 작성

In [81]:
# 지속 회원 추출
#  usglog 데이터 결합
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [82]:
# 지속 회원 데이터를 회원당 1개가 되게 언더샘플링
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201806,TS536483,5,8.0,XXXX,C01,M,2015-09-01,NaN,CA1,...,종일,10500.0,일반,4.916667,5.0,8.0,1.0,1.0,2019-04-30,43.0
1,201903,OA818294,6,5.0,XXXXX,C02,F,2017-01-01,NaN,CA1,...,주간,7500.0,일반,5.750000,6.0,8.0,2.0,1.0,2019-04-30,27.0
2,201902,PL351843,1,1.0,XXXXX,C01,M,2015-11-01,NaN,CA1,...,종일,10500.0,일반,5.166667,5.5,9.0,1.0,1.0,2019-04-30,41.0
3,201805,IK755601,8,6.0,XXXX,C03,F,2017-06-01,NaN,CA1,...,야간,6000.0,일반,6.000000,6.0,8.0,4.0,1.0,2019-04-30,22.0
4,201902,PL849752,6,4.0,XXX,C03,M,2016-09-01,NaN,CA1,...,야간,6000.0,일반,5.250000,5.5,7.0,2.0,1.0,2019-04-30,31.0


In [83]:
# 탈퇴회원 + 지속회원
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201806,TS536483,5,8.0,XXXX,C01,M,2015-09-01,NaT,CA1,...,10500.0,일반,4.916667,5.0,8.0,1.0,1.0,2019-04-30,43.0,NaN
1,201903,OA818294,6,5.0,XXXXX,C02,F,2017-01-01,NaT,CA1,...,7500.0,일반,5.750000,6.0,8.0,2.0,1.0,2019-04-30,27.0,NaN
2,201902,PL351843,1,1.0,XXXXX,C01,M,2015-11-01,NaT,CA1,...,10500.0,일반,5.166667,5.5,9.0,1.0,1.0,2019-04-30,41.0,NaN
3,201805,IK755601,8,6.0,XXXX,C03,F,2017-06-01,NaT,CA1,...,6000.0,일반,6.000000,6.0,8.0,4.0,1.0,2019-04-30,22.0,NaN
4,201902,PL849752,6,4.0,XXX,C03,M,2016-09-01,NaT,CA1,...,6000.0,일반,5.250000,5.5,7.0,2.0,1.0,2019-04-30,31.0,NaN


### 4 : 예측할 달의 재적기간 작성

In [84]:
# 재적기간 = 연월칼럼(now_date) - start_date
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["연월"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201806,TS536483,5,8.0,XXXX,C01,M,2015-09-01,NaT,CA1,...,4.916667,5.0,8.0,1.0,1.0,2019-04-30,43.0,NaN,33,2018-06-01
1,201903,OA818294,6,5.0,XXXXX,C02,F,2017-01-01,NaT,CA1,...,5.750000,6.0,8.0,2.0,1.0,2019-04-30,27.0,NaN,26,2019-03-01
2,201902,PL351843,1,1.0,XXXXX,C01,M,2015-11-01,NaT,CA1,...,5.166667,5.5,9.0,1.0,1.0,2019-04-30,41.0,NaN,39,2019-02-01
3,201805,IK755601,8,6.0,XXXX,C03,F,2017-06-01,NaT,CA1,...,6.000000,6.0,8.0,4.0,1.0,2019-04-30,22.0,NaN,11,2018-05-01
4,201902,PL849752,6,4.0,XXX,C03,M,2016-09-01,NaT,CA1,...,5.250000,5.5,7.0,2.0,1.0,2019-04-30,31.0,NaN,29,2019-02-01


### 5 : 결측치 제거

In [85]:
# end_date, exit_data, count_1 결측치가 있음 : end_date, exit_data의 유지 회원은 결측치임(삭제하면 안됨)
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               263
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [86]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2631
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2631
period                  0
now_date                0
dtype: int64

### 6 : 문자열 변수를 처리할수 있도록 가공

In [87]:
# 가입 캠페인 구분, 회원 구분, 성별등의 카테고리 변수 처리
# X : 1개월 전의 이용횟수(count_1), campaign_name, class_name, gender, 정기이용 여부 플래그(routine_flg), 재적기간(period)
# y :  탈퇴 플래그(is_deleted)

In [88]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,종일,M,8.0,1.0,33,0.0
1,일반,주간,F,5.0,1.0,26,0.0
2,일반,종일,M,1.0,1.0,39,0.0
3,일반,야간,F,6.0,1.0,11,0.0
4,일반,야간,M,4.0,1.0,29,0.0


In [89]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,8.0,1.0,33,0.0,True,False,False,False,True,False,False,True
1,5.0,1.0,26,0.0,True,False,False,False,False,True,True,False
2,1.0,1.0,39,0.0,True,False,False,False,True,False,False,True
3,6.0,1.0,11,0.0,True,False,False,True,False,False,True,False
4,4.0,1.0,29,0.0,True,False,False,True,False,False,False,True


In [90]:
del predict_data["campaign_name_일반"]
del predict_data["class_name_야간"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,8.0,1.0,33,0.0,False,False,True,False,False
1,5.0,1.0,26,0.0,False,False,False,True,True
2,1.0,1.0,39,0.0,False,False,True,False,False
3,6.0,1.0,11,0.0,False,False,False,False,True
4,4.0,1.0,29,0.0,False,False,False,False,False


### 7 : 의사결정나무를 사용해서 탈퇴예측모델을 작성하자

In [91]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

# 탈퇴 데이터와 유지 데이터의 개수를 정리
# 유지 데이터 : 2842개, 탈퇴 데이터 : 1104 -> 유지데이터에서 임의로 1104건을 추출해서 50:50으로 비율을 맞춤

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))
print(len(exit), len(conti))

1052 1052


In [92]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0.
 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1.
 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1.
 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1.

In [93]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
1287,0.0,0.0
1314,0.0,0.0
1846,0.0,0.0
23,1.0,1.0
863,1.0,1.0


### 8 : 예측 모델을 평가하고, 모델 튜닝

In [94]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8935361216730038


In [95]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8935361216730038
0.9828897338403042


In [96]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9334600760456274
0.9220532319391636


### 9 : 모델에 기여하고 있는 변수 확인

In [97]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

# 1개월 전의 이용 횟수, 정기 이용 여부, 재적기간이 Y값에 영향을 많이 줌

,feature_names,coefficient
0,count_1,0.350463
1,routine_flg,0.131219
2,period,0.518318
3,campaign_name_입회비무료,0.000000
4,campaign_name_입회비반액할인,0.000000
5,class_name_종일,0.000000
6,class_name_주간,0.000000
7,gender_F,0.000000


### 10 : 회원의 탈퇴 예측

In [98]:
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

In [99]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
    
input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)
input_data

[3, 1, 10, 0, 1, 1, 0, 0]

In [100]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0.03448276 0.96551724]]
